In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import pandas as pd

Using TensorFlow backend.


In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dataset2.json to dataset2.json
User uploaded file "dataset2.json" with length 1070528 bytes


In [3]:
def loadCSV():
    train = pd.read_json("dataset2.json")
    train= train.T
    print ('dataset cargado con "shape"', train.shape)
    return train

train = loadCSV()
train.head(10)

dataset cargado con "shape" (5889, 2)


,label,tweet
0,OBAMA,"“Low plastic stool, cheap but delicious noodle..."
1,OBAMA,"“Low plastic stool, cheap but delicious noodle..."
10,OBAMA,Michelle and I are so inspired by all the youn...
100,OBAMA,Senate leaders' obstruction of our Supreme Cou...
1000,OBAMA,"In 31 states, people can still be fired becaus..."
1001,OBAMA,"Seven down, one more to go. Make sure you tune..."
1002,OBAMA,Have you entered yet? You and a friend could w...
1003,OBAMA,Check out five big victories that made 2015 a ...
1004,OBAMA,Suit up—it's President Obama's final State of ...
1005,OBAMA,You can still join the team fighting for progr...


In [0]:
train = train.drop(train.index[1]) # Borramos primera fila que está repetida

In [5]:
obama = train.loc[train['label'] == 'OBAMA']
obama.head(10)

,label,tweet
0,OBAMA,"“Low plastic stool, cheap but delicious noodle..."
10,OBAMA,Michelle and I are so inspired by all the youn...
100,OBAMA,Senate leaders' obstruction of our Supreme Cou...
1000,OBAMA,"In 31 states, people can still be fired becaus..."
1001,OBAMA,"Seven down, one more to go. Make sure you tune..."
1002,OBAMA,Have you entered yet? You and a friend could w...
1003,OBAMA,Check out five big victories that made 2015 a ...
1004,OBAMA,Suit up—it's President Obama's final State of ...
1005,OBAMA,You can still join the team fighting for progr...
1006,OBAMA,Seeking adventure? Find it in Alaska. Enter @O...


In [0]:
text = ''
for index, row in obama.iterrows():
    text = text + ' ' + row['tweet']

In [7]:
text

' “Low plastic stool, cheap but delicious noodles, cold Hanoi beer.” This is how I’ll remember Tony. He taught us about food — but more importantly, about its ability to bring us together. To make us a little less afraid of the unknown. We’ll miss him. https://t.co/orEXIaEMZM Michelle and I are so inspired by all the young people who made today’s marches happen. Keep at it. You’re leading us forward. Nothing can stand in the way of millions of voices calling for change. Senate leaders\' obstruction of our Supreme Court nomination process is unacceptable. Say so: https://t.co/t3pYMO8Fp0 #DoYourJob In 31 states, people can still be fired because of who they love. Help change that in 2016: https://t.co/19OA1ZV27Y https://t.co/FPJE5lqg3R Seven down, one more to go. Make sure you tune in for President Obama\'s last #SOTU: https://t.co/HFgZR9WXRC https://t.co/xqSZBhULy2 Have you entered yet? You and a friend could win a trip to Alaska: https://t.co/t6XiS36KEa https://t.co/WOixbqA6Tb Check ou

In [0]:
import re

#The following is done to tokenize the tweets into its appropriate form
#In particular, we try to capture some emoticons, HTML tags, Twitter @usernames (@-mentions), Twitter #hashtags, 
#URLs, numbers, words with and without dashes and apostrophes

#Source : https://marcobonzanini.com/2015/03/09/mining-twitter-data-with-python-part-2/
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [0]:
tokens = preprocess(text)

for index,element in enumerate(tokens):
            
            #Removing '#' 
            if('#' in element):
                
                del tokens[index]
                text = text + ""
                continue
            
            
            #Removing the 'RT' tag
            elif('RT' in element):
                
                del tokens[index]
                text = text + ""
                continue
            
            #This character usually follows the 'RT' tag, so we remove it
            elif(':' in element):
                
                del tokens[index]
                text = text + ""
                continue
                
            text = text + " " + tokens[index]
        #text = text + '\n'

In [10]:
text

' “Low plastic stool, cheap but delicious noodles, cold Hanoi beer.” This is how I’ll remember Tony. He taught us about food — but more importantly, about its ability to bring us together. To make us a little less afraid of the unknown. We’ll miss him. https://t.co/orEXIaEMZM Michelle and I are so inspired by all the young people who made today’s marches happen. Keep at it. You’re leading us forward. Nothing can stand in the way of millions of voices calling for change. Senate leaders\' obstruction of our Supreme Court nomination process is unacceptable. Say so: https://t.co/t3pYMO8Fp0 #DoYourJob In 31 states, people can still be fired because of who they love. Help change that in 2016: https://t.co/19OA1ZV27Y https://t.co/FPJE5lqg3R Seven down, one more to go. Make sure you tune in for President Obama\'s last #SOTU: https://t.co/HFgZR9WXRC https://t.co/xqSZBhULy2 Have you entered yet? You and a friend could win a trip to Alaska: https://t.co/t6XiS36KEa https://t.co/WOixbqA6Tb Check ou

In [11]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 102


In [12]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 140
step = 1
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 578238


In [13]:
len(sentences[2])

140

In [14]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [15]:
# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [17]:
# train the model, output generated text after each iteration
for iteration in range(1, 5):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=2)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        print('\nGenerated')
        sys.stdout.write(generated)

        for i in range(20):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1


/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/2
578238/578238 [==============================] - 1515s 3ms/step - loss: 1.5927
Epoch 2/2
578238/578238 [==============================] - 1515s 3ms/step - loss: 1.3186

----- diversity: 0.2
----- Generating with seed: " care marketplace is open for business—check out your options today: http://t.co/FOlpuRNavS #ReadySetEnroll Let's go: The health care market"

Generated
 care marketplace is open for business—check out your options today: http://t.co/FOlpuRNavS #ReadySetEnroll Let's go: The health care marketplace the facts of t

----- diversity: 0.5
----- Generating with seed: " care marketplace is open for business—check out your options today: http://t.co/FOlpuRNavS #ReadySetEnroll Let's go: The health care market"

Generated
 care marketplace is open for business—check out your options today: http://t.co/FOlpuRNavS #ReadySetEnroll Let's go: The health care marketplace the months of 

----- diversity: 1.0
----- Generating with seed: " care marketplace is open for business—ch